# Different Class Different Color-Size-Texture (DCDCST) Comparison - SyntheticKonkle

This notebook compares CVCL and CLIP models on prototype evaluation using the SyntheticKonkle dataset.
The task is 4-way classification where distractors are from different classes AND have different color, size, and texture.

In [1]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------------------------ --------- 9.7/12.8 MB 75.5 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 44.6 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import os
import sys
import random
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from collections import defaultdict

# Path setup
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir))

# Add discover-hidden-visual-concepts to path
DISCOVER_ROOT = os.path.join(REPO_ROOT, 'discover-hidden-visual-concepts')
sys.path.insert(0, DISCOVER_ROOT)
sys.path.insert(0, REPO_ROOT)

# Import from discover-hidden-visual-concepts repo
sys.path.append(os.path.join(DISCOVER_ROOT, 'src'))
from utils.model_loader import load_model
from models.feature_extractor import FeatureExtractor
from models.multimodal.multimodal_lit import MultiModalLitModel

# SyntheticKonkle paths
DATA_DIR = os.path.join(REPO_ROOT, 'data', 'SyntheticKonkle')
MASTER_CSV = os.path.join(REPO_ROOT, 'PatrickProject', 'Chart_Generation', 'synthetic_prototype_results.csv')

c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\clip\clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


In [3]:
# Build combined dataset from all class folders
def build_synthetic_dataset():
    """Combine all labels.csv files from class_color folders."""
    all_data = []
    
    # Get all class folders (ignore _bases folders)
    class_folders = [d for d in os.listdir(DATA_DIR) 
                    if os.path.isdir(os.path.join(DATA_DIR, d)) 
                    and d.endswith('_color')]
    
    for folder in class_folders:
        labels_path = os.path.join(DATA_DIR, folder, 'labels.csv')
        if os.path.exists(labels_path):
            df = pd.read_csv(labels_path)
            df['folder'] = folder  # Track which folder the image is in
            all_data.append(df)
    
    combined_df = pd.concat(all_data, ignore_index=True)
    # Remove any rows with NaN in required columns
    combined_df = combined_df.dropna(subset=['class', 'color', 'size', 'texture'])
    print(f"Loaded {len(combined_df)} images from {len(class_folders)} classes")
    unique_classes = combined_df['class'].unique()
    print(f"Classes: {sorted([c for c in unique_classes if isinstance(c, str)])[:10]}...")  # Show first 10
    print(f"Total images in dataset: {len(combined_df)}")
    return combined_df

class SyntheticCSTImageDataset(Dataset):
    """Dataset for SyntheticKonkle images with color, size, and texture."""
    def __init__(self, df, data_dir, transform):
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Image path: data_dir/class_color/filename
        img_path = os.path.join(self.data_dir, row['folder'], row['filename'])
        try:
            img = Image.open(img_path).convert('RGB')
            return self.transform(img), row['class'], row['color'], row['size'], row['texture'], idx
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a black image as fallback
            img = Image.new('RGB', (224, 224), color='black')
            return self.transform(img), row['class'], row['color'], row['size'], row['texture'], idx

def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch])
    classes = [b[1] for b in batch]
    colors = [b[2] for b in batch]
    sizes = [b[3] for b in batch]
    textures = [b[4] for b in batch]
    idxs = [b[5] for b in batch]
    return imgs, classes, colors, sizes, textures, idxs

def run_dcdcst_test(model_name, seed=0, device='cuda' if torch.cuda.is_available() else 'cpu', 
                    batch_size=64, trials_per_combo=10, max_trials=4000):
    """Run Different Class Different Color-Size-Texture (DCDCST) evaluation on SyntheticKonkle.
    
    Tests if model can identify objects when distractors differ in class, color, size, AND texture.
    Distractors must have ALL four attributes different from the query.
    """
    random.seed(seed)
    torch.manual_seed(seed)

    # 1) Load model & transform
    model, transform = load_model(model_name, seed=seed, device=device)
    extractor = FeatureExtractor(model_name, model, device)
    print(f"[INFO] Loaded model '{model_name}'")

    # 2) Build dataset and extract embeddings
    df = build_synthetic_dataset()
    ds = SyntheticCSTImageDataset(df, DATA_DIR, transform)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)

    all_embs, all_meta, all_idxs = [], [], []
    with torch.no_grad():
        for imgs, classes, colors, sizes, textures, idxs in loader:
            feats = extractor.get_img_feature(imgs.to(device))
            feats = extractor.norm_features(feats).cpu()
            feats = feats.float()
            all_embs.append(feats)
            all_meta.extend(zip(classes, colors, sizes, textures))
            all_idxs.extend(idxs)
    all_embs = torch.cat(all_embs, dim=0)
    print(f"[INFO] Extracted embeddings for {len(all_idxs)} images")

    # 3) Group by (class, color, size, texture) quadruple
    group_map = defaultdict(list)
    for idx, (cls, col, sz, tex) in zip(all_idxs, all_meta):
        group_map[(cls, col, sz, tex)].append(idx)

    # 4) Calculate combinations and trials
    all_combinations = []
    for (cls, col, sz, tex), idx_list in group_map.items():
        # DCDCST: Get indices where ALL four attributes are different
        pool = []
        for (c2, co2, sz2, tex2), other_idxs in group_map.items():
            if c2 != cls and co2 != col and sz2 != sz and tex2 != tex:  # All four must be different
                pool.extend(other_idxs)
        
        if len(idx_list) >= 1 and len(pool) >= 3:
            all_combinations.append(((cls, col, sz, tex), idx_list, pool))
    
    # Adjust trials to stay under max_trials
    total_combinations = len(all_combinations)
    if total_combinations * trials_per_combo > max_trials:
        trials_per_group = max(1, max_trials // total_combinations)
        print(f"[INFO] Limiting to {trials_per_group} trials per combination to stay under {max_trials} total trials")
    else:
        trials_per_group = trials_per_combo
    
    # 5) Run evaluation
    total_correct = 0
    total_trials = 0
    results = {}
    
    print(f"[INFO] Running 4-way DCDCST trials ({total_combinations} combinations)")
    print("[INFO] Distractors: different class AND different color AND different size AND different texture")
    
    for (cls, col, sz, tex), idx_list, pool in all_combinations:
        if total_trials >= max_trials:
            print(f"[INFO] Reached maximum trials limit ({max_trials})")
            break
            
        correct = 0
        actual_trials = min(trials_per_group, max_trials - total_trials)
        
        for _ in range(actual_trials):
            # Pick query
            q = random.choice(idx_list)
            
            # Build prototype from other examples in same group
            same_group = [i for i in idx_list if i != q]
            if same_group:
                proto = all_embs[[all_idxs.index(i) for i in same_group]].mean(0)
            else:
                proto = all_embs[all_idxs.index(q)]
            proto = proto / proto.norm()

            # Pick 3 distractors
            distractors = random.sample(pool, 3)
            candidates = [q] + distractors
            
            # Compute similarities and predict
            feats_cand = all_embs[[all_idxs.index(i) for i in candidates]]
            sims = feats_cand @ proto
            guess = candidates[sims.argmax().item()]

            correct += int(guess == q)
            total_correct += int(guess == q)
            total_trials += 1

        acc = correct / actual_trials if actual_trials > 0 else 0
        results[f"{cls}-{col}-{sz}-{tex}"] = {
            'correct': correct,
            'trials': actual_trials,
            'accuracy': acc
        }
        
        if total_trials % 100 == 0:
            print(f"Progress: {total_trials}/{max_trials} trials completed")

    overall_acc = total_correct / total_trials if total_trials else 0.0
    print(f"\n[OK] Overall accuracy: {total_correct}/{total_trials} ({overall_acc:.1%})")
    
    # 6) Save results
    summary_df = pd.DataFrame([{
        'Model': model_name,
        'Test': 'DCDCST-SyntheticKonkle',
        'Correct': total_correct,
        'Trials': total_trials,
        'Accuracy': overall_acc
    }])
    
    os.makedirs(os.path.dirname(MASTER_CSV), exist_ok=True)
    if os.path.exists(MASTER_CSV):
        summary_df.to_csv(MASTER_CSV, mode='a', header=False, index=False, float_format='%.4f')
    else:
        summary_df.to_csv(MASTER_CSV, index=False, float_format='%.4f')

    return results, overall_acc

## CVCL Test

In [4]:
# Run CVCL evaluation
cvcl_results, cvcl_overall = run_dcdcst_test('cvcl-resnext')

print("\nCVCL Results (sample):")
for i, (key, res) in enumerate(cvcl_results.items()):
    if i < 20:  # Show first 20
        print(f"{key:40s}: {res['correct']}/{res['trials']} ({res['accuracy']:.1%})")
print(f"\nCVCL Overall Accuracy: {cvcl_overall:.1%}")

Loading checkpoint from C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt


Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt`


[INFO] Loaded model 'cvcl-resnext'
Loaded 7691 images from 68 classes
Classes: ['abacus', 'apple', 'axe', 'babushkadolls', 'bagel', 'basket', 'bell', 'bonzai', 'bread', 'butterfly']...
Total images in dataset: 7691
Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\axe_color\axe_medium_bumpy_02_black.png: cannot identify image file 'c:\\Users\\jbats\\Projects\\NTU-Synthetic\\data\\SyntheticKonkle\\axe_color\\axe_medium_bumpy_02_black.png'
Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\axe_color\axe_small_bumpy_01_yellow.png: cannot identify image file 'c:\\Users\\jbats\\Projects\\NTU-Synthetic\\data\\SyntheticKonkle\\axe_color\\axe_small_bumpy_01_yellow.png'
Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\bread_color\bread_large_bumpy_02_brown.png: [Errno 2] No such file or directory: 'c:\\Users\\jbats\\Projects\\NTU-Synthetic\\data\\SyntheticKonkle\\bread_color\\bread_large_bumpy_02_brown.png'
E

## CLIP Test

In [5]:
# Run CLIP evaluation
clip_results, clip_overall = run_dcdcst_test('clip-resnext')

print("\nCLIP Results (sample):")
for i, (key, res) in enumerate(clip_results.items()):
    if i < 20:  # Show first 20
        print(f"{key:40s}: {res['correct']}/{res['trials']} ({res['accuracy']:.1%})")
print(f"\nCLIP Overall Accuracy: {clip_overall:.1%}")

[INFO] Loaded model 'clip-resnext'
Loaded 7304 images from 68 classes
Classes: ['abacus', 'apple', 'axe', 'babushkadolls', 'bagel', 'basket', 'bell', 'bonzai', 'breadloaf', 'calculator']...
Total images in dataset: 7304


c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\axe_color\axe_medium_bumpy_02_black.png: cannot identify image file 'c:\\Users\\jbats\\Projects\\NTU-Synthetic\\data\\SyntheticKonkle\\axe_color\\axe_medium_bumpy_02_black.png'
Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\axe_color\axe_small_bumpy_01_yellow.png: cannot identify image file 'c:\\Users\\jbats\\Projects\\NTU-Synthetic\\data\\SyntheticKonkle\\axe_color\\axe_small_bumpy_01_yellow.png'
Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\breadloaf_color\breadloaf_large_bumpy_02_brown.png: [Errno 2] No such file or directory: 'c:\\Users\\jbats\\Projects\\NTU-Synthetic\\data\\SyntheticKonkle\\breadloaf_color\\breadloaf_large_bumpy_02_brown.png'
Error loading image c:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\grill_color\grill_small_smooth_02_black.png: [Errno 2] No such file or directory: 'c:\\Users\\jbats\\Projects\\NTU-Synthe